In [1]:
import pandas as pd
import seaborn as sns
import re
from glob import glob
from collections import Counter

In [2]:
out_files = glob('../data/lrmi-crawler-out-wdc19/*.out')

In [3]:
triples = []
for out_file in out_files:
    with open(out_file, 'r') as f:
        for l in f:
            triples.append(l[:-1].split(','))

In [4]:
len(triples)

1817094

In [5]:
triples = [t for t in triples if len(t) == 4]

In [6]:
df = pd.DataFrame(triples, columns=['s', 'p', 'o', 'u'])

In [7]:
df['simple_p'] = df.p.str.extract('.*/([a-zA-Z]+)>')[0]

In [8]:
df['domain'] = df.u.str.extract('<http.*://(.*?)/.*>')[0]

In [9]:
df.domain.value_counts()

www.litcharts.com           115870
www.tabletennis365.com       43873
www.math-drills.com          38390
www.ck12.org                 36820
tabletennis365.com           34606
                             ...  
zrfx7j.rifengwang.top            1
wenming-cn.kanwangju.com         1
police.news.sohu.com             1
77sbcp.com                       1
m.qyssk.com.cn                   1
Name: domain, Length: 4572, dtype: int64

In [10]:
import requests 
from tqdm import tqdm

texts = []
for website in tqdm(Out[9][Out[9] > Out[9].quantile(.9)].index):
    try:
        texts.append(requests.get('http://' + website, timeout=10).text())
    except:
        pass

100%|██████████| 458/458 [1:19:51<00:00, 10.46s/it]     


In [11]:
df.simple_p.value_counts()

timeRequired            403042
typicalAgeRange         299452
learningResourceType    284225
educationalUse          232838
educationalRole         188879
isBasedOnUrl            158725
interactivityType        72557
alignmentType            44757
targetName               35820
educationalAlignment     33069
educationalFramework     19328
targetDescription        15070
useRightsURL             10204
targetURL                 1515
educationalLevel           389
name                        16
aggregateRating             16
Name: simple_p, dtype: int64

In [37]:
src_p = df.groupby(['domain', 'simple_p']).u.count().rename('count').unstack(level=1)

In [38]:
src_p['sum'] = src_p.sum(axis=1)

In [39]:
top = src_p.sort_values(by='sum', ascending=False).iloc[:10]

In [41]:
top

simple_p,isBasedOnUrl,timeRequired,typicalAgeRange,learningResourceType,alignmentType,educationalAlignment,educationalRole,targetName,interactivityType,targetURL,educationalUse,educationalFramework,useRightsURL,educationalLevel,targetDescription,aggregateRating,sum
domain,,,,,,,,,,,,,,,,,
www.litcharts.com,NaN,NaN,NaN,6.0,NaN,NaN,115864.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115870.0
www.tabletennis365.com,43873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43873.0
www.math-drills.com,NaN,NaN,NaN,NaN,NaN,NaN,23034.0,NaN,NaN,NaN,15356.0,NaN,NaN,NaN,NaN,NaN,38390.0
www.ck12.org,NaN,NaN,11202.0,6290.0,NaN,NaN,NaN,NaN,10698.0,NaN,NaN,NaN,8630.0,NaN,NaN,NaN,36820.0
tabletennis365.com,34606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34606.0
www.getabstract.com,NaN,16063.0,NaN,16063.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32126.0
www.commonsense.org,NaN,52.0,NaN,72.0,7740.0,7740.0,7518.0,6224.0,NaN,NaN,1455.0,989.0,NaN,NaN,NaN,NaN,31790.0
www.sondakika.com,NaN,12677.0,12677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25354.0
www.domkino.tv,NaN,NaN,22330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22330.0
